In [ ]:
import pandas as pd
import numpy as np
import embedder as emb

In [ ]:
selling_time_dict ={"un'ora": 60,
                    }

In [ ]:
df_sold = pd.read_csv('data.csv')
df_not_sold = pd.read_csv('data.csv')

df_final = pd.concat([df_sold, df_not_sold], axis=0)

In [ ]:
columns_to_drop = ["Dataid", "Images", "Link", "SearchDate", "SellerId", "SellerName", "Condition", "Page", "SearchCount"]

df_final = df_final.drop(columns=columns_to_drop)

for index, row in df_final.iterrows():
    if "min" in row["Upload_date"]:
        df_final.at[index, "Upload_date"] = int(row["Upload_date"].split(" ")[0])
    elif row["Upload_date"] in selling_time_dict:
        df_final.at[index, "Upload_date"] = selling_time_dict[row["Upload_date"]]
    else:
        print("Error: ", row["Upload_date"])
        df_final = df_final.drop(index)


In [ ]:
df_final['MarketStatus'] = df_final['MarketStatus'].map({'On Sale': 0, 'Sold': 1})

# Move 'MarketStatus' column to the end
market_status = df_final.pop('MarketStatus')
df_final['MarketStatus'] = market_status

In [ ]:
sold_descriptions = df_final[df_final['MarketStatus'] == 1]['Description'].tolist()
not_sold_descriptions = df_final[df_final['MarketStatus'] == 0]['Description'].tolist()

In [ ]:
# Getting embeddings
sold_embeddings = [emb.get_embedding(sentence) for sentence in sold_descriptions]
not_sold_embeddings = [emb.get_embedding(sentence) for sentence in not_sold_descriptions]

# Calculating average locations (centroids)
sold_centroid = emb.calculate_centroid(sold_embeddings)
not_sold_centroid = emb.calculate_centroid(not_sold_embeddings)